<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

# Erste Schritte mit TensorBoard

Um beim Vorgänge beim maschinellen Lernen besser verstehen zu können, sollte man Kennzahlen messen und viosualisieren. TensorBoard ist ein Werkzeug, mit dem man solches Messungen und Visualisierungen während des maschinellen Lernprozesses umsetzen kann. Es ermöglicht die Verfolgung von Metriken wie Verlust und Genauigkeit, die Visualisierung des Modellgraphen, die Projektion von Einbettungen in einen niedrigdimensionalen Raum und vieles mehr.

Dieser Schnellstart zeigt, wie man schnell mit TensorBoard loslegen kann. Die verbleibenden Anleitungen auf dieser Website bieten mehr Details über spezifische Fähigkeiten, von denen viele hier nicht enthalten sind.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Wirt verwenden erneut den [MNIST](https://en.wikipedia.org/wiki/MNIST_database)-Datensatz als Beispiel.
Als erstes normalisieren wir die Daten und schreiben Sie eine Funktion, die ein einfaches Keras-Modell zur Klassifizierung der Bilder in 10 Klassen erstellt.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
        tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
        tf.keras.layers.Dropout(0.2, name='layers_dropout'),
        tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
    ])

## TensorBoard mit der `fit`-Methodes eines Keras Modells verwenden

Wenn wir ein Modell mit [Model.fit()] (https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit) aus Keras trainieren, fügen wir den Callback `tf.keras.callbacks.TensorBoard` hinzu. Dies führt dazu, dass Protokolle erstellt und gespeichert werden. Wir aktivieren zusätzlich die Histogrammberechnung in jeder Epoche mit `histogram_freq=1`.

Es ist sinnvoll, die Logs in einem Unterverzeichnis anzulege, dass mit einem Zeitstempel im Namen versehen wird. Dies erlaubt im Nachgang eine einfache Auswahl der verschiedenen Trainingsläufe.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Nun starten wir TensorBoard über die Kommandozeile oder innerhalb eines Notebooks. Die beiden Schnittstellen sind im Allgemeinen die gleichen. In Notebooks benutzen wir das `ipython` *Magic Command* `%tensorboard`. Auf der Kommandozeile würde man den gleichen Befehl ohne "%" ausführen.

In [ ]:
%tensorboard --logdir logs/fit/

Falls Sie eine Fehlermeldung der Art `ERROR: Could not find 'tensorboard'` erhalten, findet das Notebook das TensorBoard Programm nicht.
Finden Sie den Speicherort des TensorBoard Programms heraus (z.B. mit `which tensorboard` auf Linux oder MacOS Systemen) und tragen Sie entweder den Pfad in der Umgebungsvariablen `PATH` ein, oder setzen Sie explizit die Umgebungsvariable `TENSORBOARD_BINARY`. Dies können Sie auch direkt aus dem Notebook heraus bewerkstelligen, z.B. so:
```python
import os
os.environ["TENSORBOARD_BINARY"]="/opt/homebrew/Caskroom/miniforge/base/bin/tensorboard"
```

In [ ]:
import os
os.environ["TENSORBOARD_BINARY"]="/opt/homebrew/Caskroom/miniforge/base/bin/tensorboard"

Ein kurzer Überblick über die in diesem Beispiel erstellten Visualisierungen und die Dashboards (Registerkarten in der oberen Navigationsleiste), in denen sie zu finden sind:

* **Skalare** zeigen, wie sich der Verlust und die Metriken mit jeder Epoche ändern. Sie können sie auch verwenden, um die Trainingsgeschwindigkeit, die Lernrate und andere skalare Werte zu verfolgen. Skalare sind in den Dashboards **Time Series** oder **Scalars** zu finden.
* **Graphen** helfen Ihnen bei der Visualisierung Ihres Modells. In diesem Fall wird das Keras-Diagramm der Schichten angezeigt, was Ihnen helfen kann, sicherzustellen, dass es korrekt aufgebaut ist. Graphen finden Sie im Dashboard **Graphs**.
* **Histogramme** und **Verteilungen** zeigen die Verteilung eines Tensors über die Zeit. Dies kann nützlich sein, um Gewichte und Verzerrungen zu visualisieren und zu überprüfen, ob sie sich in der erwarteten Weise verändern. Histogramme sind in den Dashboards **Time Series** oder **Histogramms** zu finden. Verteilungen können im **Distributions** Dashboard gefunden werden.

Zusätzliche TensorBoard Dashboards werden automatisch aktiviert, wenn Sie andere Datentypen protokollieren. Zum Beispiel können Sie mit dem Keras TensorBoard Callback auch Bilder und Einbettungen protokollieren. Sie können sehen, welche anderen Dashboards in TensorBoard verfügbar sind, indem Sie auf das "inaktive" Dropdown oben rechts klicken.

## Trainingsläufe mit TensorBoard.dev öffentlich hosten

[TensorBoard.dev](https://tensorboard.dev) ist ein kostenloser öffentlicher Dienst, der es Ihnen ermöglicht, Ihre TensorBoard-Protokolle hochzuladen und einen Permalink zu erhalten, der in Ausarbeitungen, Blogbeiträgen, sozialen Medien usw. öffentlich geteilt werden kann. Dies kann eine bessere Reproduzierbarkeit und Zusammenarbeit ermöglichen.

Um TensorBoard.dev zu benutzen, führen Sie den folgenden Befehl aus:


In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

Beachten Sie, dass dieser Aufruf das Ausrufepräfix (`!`) verwendet, um die Shell aufzurufen, und nicht das Prozent-Präfix (`%`), um ein *Magic Command* aufzurufen.  Wenn Sie diesen Befehl von der Kommandozeile aus aufrufen, ist keines der beiden Präfixe erforderlich.

Sehen Sie sich ein Beispiel an [hier](https://tensorboard.dev/experiment/EDZb7XgKSBKo6Gznh3i8hg/#scalars).

Für weitere Details über die Verwendung von TensorBoard.dev, siehe https://tensorboard.dev/#get-started

## Referenzen

Das Beispiel wurde übernommen aus dem Notebook [Get started with TensorBoard](https://www.tensorflow.org/tensorboard/get_started) der [TensorFlow Dokumentation](https://www.tensorflow.org) und unterliegt der [Apache Lizenz 2.0](https://www.apache.org/licenses/LICENSE-2.0)